In [ ]:
!pip install cplex
!pip install docplex


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671350 sha256=61089dd9b819f75513bc718fe92aa80c7fe40ad18661396b4a1612cc8b764d5e
  Stored in directory: /root/.cache/pip/wheels/3b/e5/00/0bf0173d67188fe73a13e3a61412b3f975f60205e3fab93a69
Successfully built docplex


In [ ]:
import pandas as pd
from docplex.cp.model import CpoModel, CpoSolver, context
import cplex
import numpy as np
import math

from itertools import chain


In [ ]:
# compute the multiplication tensor of A: m by n and B: n by p
def general_multiplication_tensor(N, M, P):
   """Multiplication tensor.
   The multiplication tensor T in {0,1} of size NM x MP x PN
   for the multiplication of two matrices of size NxM and MxP
   """
   T = np.zeros((N * M, M * P, N * P), dtype=np.int64)

   for n in range(N):
       for m in range(M):
           for p in range(P):
               # Convert multi-dimensional indices to flat indices.
               a_index = np.ravel_multi_index((n, m), (N, M))
               b_index = np.ravel_multi_index((m, p), (M, P))
               c_index = np.ravel_multi_index((n, p), (N, P))
               T[a_index, b_index, c_index] = 1

   return T

In [ ]:
# Get multiplication tensor from the factor matrices U, V, and W
def expand_pd(U, V, W):
    """Expand a polyadic decomposition.
    The polyadic expansion T of the factor matrices U, V, and W is defined by:
        T[i, j, k] = \sum_r U[i, r] * V[j, r] * W[k, r].
    """
    I, J, K, R = U.shape[0], V.shape[0], W.shape[0], U.shape[1]
    T = np.zeros((I, J, K))
    for i in range(I):
        for j in range(J):
            for k in range(K):
                for r in range(R):
                    T[i, j, k] += U[i, r] * V[j, r] * W[k, r]
    return T


In [ ]:
N = 2
M = 2
P = 2
T = general_multiplication_tensor(N,M,P)
T

array([[[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0]],

       [[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]])

In [ ]:
def CP_min_variable(N,M,P,T,R,pen_bound,optr=False,seed=4,time_limit=300):
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(R)] for i in range(N*M)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(R)] for j in range(M*P)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(R)] for k in range(N*P)]
    penalties = [[[mdl.integer_var(0, pen_bound, name="pen" + str(i) + "_" + str(j) + "_" + str(k)) for k in range(N*P)] for j in range(M*P)] for i in range(N*M)]

    # Matrix Multiplication as tensor operation
    # only add constraints where rhs is 1
    penalty = 0
    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                if T[i][j][k] == 1:
                    mdl.add(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) == T[i][j][k])
                else:
                    mdl.add(penalties[i][j][k] == mdl.abs(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R))))
                    penalty += penalties[i][j][k] #* (1.0/(pen_bound*N*N*M*M*P*P))#mdl.abs(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)))

    if optr:
      usedr = [mdl.integer_var(0,1, name="usedr" + str(r)) for r in range(R)]
      for r in range(R):
        mdl.add(mdl.sum(mdl.abs(u[i][r]) for i in range(len(u))) <= len(u)*usedr[r])
        penalty += usedr[r]

    # objective penalize zero violations
    mdl.add(mdl.minimize(penalty))

    # solve
    mdl.set_parameters({'RandomSeed':seed, 'LogPeriod': 100000})

    msol = mdl.solve(TimeLimit=time_limit) #, SolutionLimit = 1)
    if msol:
      return [msol,u,v,w]
    else:
      print("Infeasible")
      return -1

In [ ]:
def CP_min(N,M,P,T,R,seed=4, time_limit=300):
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(R)] for i in range(N*M)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(R)] for j in range(M*P)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(R)] for k in range(N*P)]

    # Matrix Multiplication as tensor operation
    # only add constraints where rhs is 1
    penalty = 0
    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                if T[i][j][k] == 1:
                    mdl.add(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) == T[i][j][k])
                else:
                    penalty += mdl.abs(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) - T[i][j][k])


    # objective penalize zero violations
    mdl.add(mdl.minimize(penalty))

    # solve
    mdl.set_parameters({'RandomSeed':seed, 'LogPeriod': 100000})

    msol = mdl.solve(TimeLimit=time_limit) #, SolutionLimit = 1)
    if msol:
      return [msol,u,v,w]
    else:
      print("Infeasible")
      return -1

In [ ]:
def CP_min_square(N,M,P,T,R,seed=4, time_limit=300):
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(R)] for i in range(N*M)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(R)] for j in range(M*P)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(R)] for k in range(N*P)]

    # Matrix Multiplication as tensor operation
    # only add constraints where rhs is 1
    penalty = 0
    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                if T[i][j][k] == 1:
                    mdl.add(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) == T[i][j][k])
                else:
                    penalty += (mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)))**2


    # objective penalize zero violations
    mdl.add(mdl.minimize(penalty))

    # solve
    mdl.set_parameters({'RandomSeed':seed, 'LogPeriod': 100000})

    msol = mdl.solve(TimeLimit=time_limit) #, SolutionLimit = 1)
    if msol:
      return [msol,u,v,w]
    else:
      print("Infeasible")
      return -1

In [ ]:
def CP_original(N,M,P,T,R,seed=4, time_limit=300):
    mdl = CpoModel()
    u = [[mdl.integer_var(-1, 1, name="U" + str(i) + "_" + str(r)) for r in range(R)] for i in range(N*M)]
    v = [[mdl.integer_var(-1, 1, name="V" + str(j) + "_" + str(r)) for r in range(R)] for j in range(M*P)]
    w = [[mdl.integer_var(-1, 1, name="W" + str(k) + "_" + str(r)) for r in range(R)] for k in range(N*P)]

    # Matrix Multiplication as tensor operation
    # only add constraints where rhs is 1
    # penalty = 0
    for i in range(N*M):
        for j in range(M*P):
            for k in range(N*P):
                # if T[i][j][k] == 1:
                mdl.add(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) == T[i][j][k])
                # else:
                #     penalty += mdl.abs(mdl.sum(u[i][r]*v[j][r]*w[k][r] for r in range(R)) - T[i][j][k])


    # objective penalize zero violations
    # mdl.add(mdl.minimize(penalty))

    # solve
    mdl.set_parameters({'RandomSeed':seed, 'LogPeriod': 100000})

    msol = mdl.solve(TimeLimit=time_limit) #, SolutionLimit = 1)
    if msol:
      return [msol,u,v,w]
    else:
      print("Infeasible")
      return -1

In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 2
P = 2
T = general_multiplication_tensor(N,M,P)
R = 7
solution = CP_min(N,M,P,T,R, seed=2)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 84 variables, 8 constraints
 ! TimeLimit            = 300
 ! LogPeriod            = 100000
 ! RandomSeed           = 2
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 133.1 (before), 133.1 (after)
 !  . Memory usage      : 577.8 kB (before), 577.8 kB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         84                 -
 + New bound is 0
 *           146     1433  0.05s        1      (gap is 100.0%)
 *            52     1508  0.05s        1      (gap is 100.0%)
 *            50     1198  0.05s        2      (gap is 100.0%)
 *            38     1625  0.05s        2      (gap is 100.0%)
 *            30     1940  0.06s        1      (gap is 100.0%)
 *           

In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 3
P = 2
T = general_multiplication_tensor(N,M,P)
R = 11
solution = CP_min_variable(N,M,P,T,R,pen_bound=2,optr=True,seed=6)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 319 variables, 155 constraints
 ! TimeLimit            = 300
 ! LogPeriod            = 100000
 ! RandomSeed           = 6
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 499.2 (before), 499.2 (after)
 !  . Memory usage      : 1.1 MB (before), 1.1 MB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        319                 -
 + New bound is 0
                        0        319    1            -
 + New bound is 1
 *           112    28670  0.67s        1      (gap is 99.11%)
 *            84    44689  0.99s        2      (gap is 98.81%)
 *            83    45746  1.05s        1      (gap is 98.80%)
 *            82    47043  1.08s        1      (gap is 98.78%)
 *  

In [ ]:
# Print solution
if type(solution)!=int:
    sol,U,V,W = solution
    U_sol = np.zeros((M*N,R))
    V_sol = np.zeros((P*M,R))
    W_sol = np.zeros((P*N,R))

    for i in range(M*N):
        for r in range(R):
            U_sol[i,r] = sol[U[i][r]]
    for i in range(P*M):
        for r in range(R):
            V_sol[i,r] = sol[V[i][r]]
    for i in range(P*N):
        for r in range(R):
            W_sol[i,r] = sol[W[i][r]]

    print('U,V,W')
    print(U_sol)
    print(V_sol)
    print(W_sol)

    print("\n\n")
    t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)

    print('T')
    print(T)
    print("\n")
    print('T_CP')
    print(t_constraint_programming)

    print("\n\n")
    # check if solution is correct
    if (t_constraint_programming==T).all():
      print("CP got the correct T")

U,V,W
[[ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]]
[[-1. -1.  1. -1.  0. -1. -1.  1.]
 [ 1.  1.  1.  0. -1. -1. -1.  1.]
 [-1.  0.  1. -1.  1. -1.  1. -1.]
 [ 0. -1. -1.  0.  1.  1.  1. -1.]]
[[ 0. -1.  1. -1.  0.  0. -1. -1.]
 [ 1.  1. -1.  0.  0.  1. -1. -1.]
 [-1. -1.  0.  1.  1.  0.  1. -1.]
 [-1.  1. -1. -1. -1.  1.  1.  1.]]



T
[[[1 0 0 0]
  [0 1 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [1 0 0 0]
  [0 1 0 0]]

 [[0 0 1 0]
  [0 0 0 1]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 1 0]
  [0 0 0 1]]]


T_CP
[[[ 1.  1. -1. -1.]
  [ 1.  1. -1. -1.]
  [-1. -1.  1.  1.]
  [-1. -1.  1.  1.]]

 [[-1. -1.  1.  1.]
  [-1. -1.  1.  1.]
  [ 1.  1. -1. -1.]
  [ 1.  1. -1. -1.]]

 [[-1. -1.  1.  1.]
  [-1. -1.  1.  1.]
  [ 1.  1. -1. -1.]
  [ 1.  1. -1. -1.]]

 [[ 1.  1. -1. -1.]
  [ 1.  1. -1. -1.]
  [-1. -1.  1.  1.]
  [-1. -1.  1.  1.]]]





In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 2
P = 3
T = general_multiplication_tensor(N,M,P)
R = 11
solution = CP_min(N,M,P,T,R, seed=4)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 176 variables, 12 constraints
 ! TimeLimit            = 600
 ! LogPeriod            = 100000
 ! RandomSeed           = 4
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 279.0 (before), 279.0 (after)
 !  . Memory usage      : 988.7 kB (before), 988.7 kB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        176                 -
 + New bound is 0
 *           364     1604  0.09s        1      (gap is 100.0%)
 *           168     1857  0.09s        2      (gap is 100.0%)
 *           131     1769  0.11s        1      (gap is 100.0%)
 *           129     5101  0.20s        2      (gap is 100.0%)
 *           127    14831  0.55s        1      (gap is 100.0%)
 *         

In [ ]:
# Print solution
if type(solution)!=int:
    sol,U,V,W = solution
    U_sol = np.zeros((M*N,R))
    V_sol = np.zeros((P*M,R))
    W_sol = np.zeros((P*N,R))

    for i in range(M*N):
        for r in range(R):
            U_sol[i,r] = sol[U[i][r]]
    for i in range(P*M):
        for r in range(R):
            V_sol[i,r] = sol[V[i][r]]
    for i in range(P*N):
        for r in range(R):
            W_sol[i,r] = sol[W[i][r]]

    print('U,V,W')
    print(U_sol)
    print(V_sol)
    print(W_sol)

    print("\n\n")
    t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)

    print('T')
    print(T)
    print("\n")
    print('T_CP')
    print(t_constraint_programming)

    print("\n\n")
    # check if solution is correct
    if (t_constraint_programming==T).all():
      print("CP got the correct T")

U,V,W
[[ 0.  1.  0.  0. -1.  0.  0.  0. -1.  0.  0.]
 [-1.  0.  1.  0. -1.  0.  0. -1.  0. -1.  0.]
 [ 0.  0. -1. -1.  0. -1.  0.  0. -1.  0. -1.]
 [ 0.  0.  0. -1.  0.  0. -1. -1.  0.  0.  0.]]
[[ 0.  1.  0.  0.  0.  0.  0.  0. -1.  0.  1.]
 [ 0. -1. -1.  0.  1.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.]
 [-1.  0.  1. -1.  0.  0.  0. -1.  0. -1. -1.]
 [ 1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0. -1.  1.  0.  0.  1.]]
[[ 0.  1.  0.  0. -1.  0.  0.  0.  0.  1.  0.]
 [-1.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.]
 [ 0.  1. -1.  0.  0.  1.  0.  0. -1.  1. -1.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  0.  0. -1.]
 [ 1.  0.  1.  1.  0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.]]



T
[[[1 0 0 0 0 0]
  [0 1 0 0 0 0]
  [0 0 1 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]]

 [[0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [0 0 0 0 0 0]
  [1 0 0 0 0 0]
  [0 1 0 0 0 0]
  [0 0 1 0 0 0]]

 [[0 0 0 1 0 0]
  [0 0 0 0 1 0]
  [0 0

In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 3
P = 2
T = general_multiplication_tensor(N,M,P)
R = 11
solution = CP_original(N,M,P,T,R, seed=1)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Satisfiability problem - 176 variables, 144 constraints
 ! TimeLimit            = 300
 ! LogPeriod            = 100000
 ! RandomSeed           = 1
 ! Initial process time : 0.04s (0.03s extraction + 0.01s propagation)
 !  . Log search space  : 279.0 (before), 279.0 (after)
 !  . Memory usage      : 949.5 kB (before), 949.5 kB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
                     100k         14    1   F    -1  = U3_9
                     100k         17    2   F    -1  = W2_10
                     200k         11    2         1 != U0_7
                     200k          3    1   F     0  = V2_1
                     300k         11    2         0  = U3_2
                     300k          9    1   F    -1 != U3_3
                     400k         1

In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 3
P = 2
T = general_multiplication_tensor(N,M,P)
R = 11
solution = CP_min_square(N,M,P,T,R, seed=1)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 176 variables, 12 constraints
 ! TimeLimit            = 300
 ! LogPeriod            = 100000
 ! RandomSeed           = 1
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 279.0 (before), 279.0 (after)
 !  . Memory usage      : 1.1 MB (before), 1.1 MB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        176                 -
 + New bound is 0
 *          1428     1532  0.09s        1      (gap is 100.0%)
 *           273     1695  0.09s        1      (gap is 100.0%)
 *           198     1838  0.09s        2      (gap is 100.0%)
 *           197     3022  0.15s        2      (gap is 100.0%)
 *           114     5102  0.21s        2      (gap is 100.0%)
 *           10

In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 3
P = 2
T = general_multiplication_tensor(N,M,P)
R = 11
solution = CP_min(N,M,P,T,R, seed=1)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 176 variables, 12 constraints
 ! TimeLimit            = 300
 ! LogPeriod            = 100000
 ! RandomSeed           = 1
 ! Initial process time : 0.03s (0.01s extraction + 0.02s propagation)
 !  . Log search space  : 279.0 (before), 279.0 (after)
 !  . Memory usage      : 1.1 MB (before), 1.1 MB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        176                 -
 + New bound is 0
 *           376     1532  0.10s        1      (gap is 100.0%)
 *           143     1695  0.10s        1      (gap is 100.0%)
 *           122     1838  0.10s        2      (gap is 100.0%)
 *            97     2615  0.14s        1      (gap is 100.0%)
 *            94    15929  0.55s        1      (gap is 100.0%)
 *            9

In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 3
P = 2
T = general_multiplication_tensor(N,M,P)
R = 11
solution = CP_min(N,M,P,T,R, seed=4)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 176 variables, 12 constraints
 ! TimeLimit            = 600
 ! LogPeriod            = 100000
 ! RandomSeed           = 4
 ! Initial process time : 0.05s (0.02s extraction + 0.03s propagation)
 !  . Log search space  : 279.0 (before), 279.0 (after)
 !  . Memory usage      : 1.1 MB (before), 1.1 MB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        176                 -
 + New bound is 0
 *           420     1852  0.20s        1      (gap is 100.0%)
 *           316     1738  0.20s        2      (gap is 100.0%)
 *           158     2018  0.25s        1      (gap is 100.0%)
 *           111     1914  0.25s        2      (gap is 100.0%)
 *            98     2834  0.30s        2      (gap is 100.0%)
 *            9

In [ ]:
# Print solution
if type(solution)!=int:
    sol,U,V,W = solution
    U_sol = np.zeros((M*N,R))
    V_sol = np.zeros((P*M,R))
    W_sol = np.zeros((P*N,R))

    for i in range(M*N):
        for r in range(R):
            U_sol[i,r] = sol[U[i][r]]
    for i in range(P*M):
        for r in range(R):
            V_sol[i,r] = sol[V[i][r]]
    for i in range(P*N):
        for r in range(R):
            W_sol[i,r] = sol[W[i][r]]

    print('U,V,W')
    print(U_sol)
    print(V_sol)
    print(W_sol)

    print("\n\n")
    t_constraint_programming = expand_pd(U_sol, V_sol, W_sol)

    print('T')
    print(T)
    print("\n")
    print('T_CP')
    print(t_constraint_programming)

    print("\n\n")
    # check if solution is correct
    if (t_constraint_programming==T).all():
      print("CP got the correct T")

U,V,W
[[ 0.  0.  0. -1.  0. -1. -1. -1.  1.  0.  1.]
 [ 0.  0. -1. -1.  0. -1.  0.  0.  0. -1.  0.]
 [ 0.  0. -1. -1.  0. -1.  0.  0.  1.  0.  0.]
 [-1.  1.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  1. -1.  0.  0.  0. -1.  0.]
 [-1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  1.]
 [ 0.  0.  0.  1.  1. -1.  0.  0. -1.  0.  0.]
 [ 0.  0.  0. -1.  0.  1.  0.  0.  1. -1.  0.]
 [ 1. -1.  0.  0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  0.  1. -1.  1. -1.]]
[[ 0.  0.  0.  0. -1.  1.  0.  1.  1.  1. -1.]
 [ 0.  0.  0.  1. -1.  1.  0.  0.  0.  1.  1.]
 [-1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -1.  1. -1. -1.  0.  0.  0. -1.]]



T
[[[1 0 0 0]
  [0 1 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [1 0 0 0]
  [0 1 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [1 0 0 0]
  [0 1 0 0]]

 [[0 0 1 0]


In [ ]:
%%time
# Run CP model for 2 by 2 matrices
N = 2
M = 2
P = 4
T = general_multiplication_tensor(N,M,P)
R = 14
solution = CP_min(N,M,P,T,R, seed=4, time_limit=1800)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 280 variables, 16 constraints
 ! TimeLimit            = 1800
 ! LogPeriod            = 100000
 ! RandomSeed           = 4
 ! Initial process time : 0.07s (0.06s extraction + 0.01s propagation)
 !  . Log search space  : 443.8 (before), 443.8 (after)
 !  . Memory usage      : 1.7 MB (before), 1.7 MB (after)
 ! Using parallel search with 2 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        280                 -
 + New bound is 0
 *           350     2312  0.31s        1      (gap is 100.0%)
 *           348     4160  0.56s        1      (gap is 100.0%)
 *           339     4879  0.56s        1      (gap is 100.0%)
 *           221     4770  0.61s        2      (gap is 100.0%)
 *           220     7259  0.74s        2      (gap is 100.0%)
 *           2